This program will download the raw IBM data and clean, and create the features that all the models will use.


In [1]:
# mounting google colab drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
import numpy as np
import pandas as pd
loc = "/content/drive/MyDrive/KE_GNN/"
os.chdir(loc)
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/KE_GNN'

Downloading IBM dataset.

In [2]:
!pip install kaggle
!kaggle datasets download -d ealtman2019/credit-card-transactions
!unzip credit-card-transactions.zip

Dataset URL: https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions
License(s): other
credit-card-transactions.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  credit-card-transactions.zip
replace User0_credit_card_transactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: User0_credit_card_transactions.csv  
replace credit_card_transactions-ibm_v2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: credit_card_transactions-ibm_v2.csv  y
y

replace sd254_cards.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: sd254_cards.csv         
replace sd254_users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: sd254_users.csv         


**LOADING IN ALL THE DATA AND DROPPING ALL FEATURES THAT WILL NOT BE USED WITHIN THE MODELS AND THESIS**

In [6]:
# transactions
transactions = pd.read_csv('{}credit_card_transactions-ibm_v2.csv'.format(loc))


# creating a payment Primary key
transactions['PK'] = transactions.index
# amount to numerical
transactions['Amount'] = pd.to_numeric(transactions['Amount'].str.replace('$',''))
# changing MCC to have 'M' prefix. This ensure that no usage or analysis will mistake the number as a continuous variable
transactions['MCC'] = "M"+transactions['MCC'].astype(str)
# changing Zip to have 'Z' prefix. This ensure that no usage or analysis will mistake the number as a continuous variable
transactions['Zip'] = "Z"+transactions['Zip'].astype(str)
#renaming Zip to Merchant_zip to avoid any confusion with Zipcode in user address
transactions.rename(columns={"Zip": "Merchant_zip"})
# changing Merchant Name to have 'MN' prefix. This ensure that no usage or analysis will mistake the number as a continuous variable
transactions['Merchant Name'] = "MN"+transactions['Merchant Name'].astype(str)


# filling na with online (when missing value is online)
transactions['Merchant State'] = transactions['Merchant State'].fillna('Online')

#users
users = pd.read_csv('{}sd254_users.csv'.format(loc))
users = users.drop(['Yearly Income - Person','Num Credit Cards','Current Age','Retirement Age'], axis = 1)
users['Total Debt'] = pd.to_numeric(users['Total Debt'].str.replace('$',''))

# creating a column with the User Primary Key
users['User'] = users.index
# changing Zip to have 'Z' prefix. This ensure that no usage or analysis will mistake the number as a continuous variable
users['Zipcode'] = "Z"+users['Zipcode'].astype(str)
users['Per Capita Income - Zipcode'] = pd.to_numeric(users['Per Capita Income - Zipcode'].str.replace('$',''))
# creating DOB assuming that everyone is born on the 1st of every month. this assumption is only to be able to create a date object and will not make any real difference
users['DOB'] = pd.to_datetime('01/' + users['Birth Month'].astype(str) + '/' + users['Birth Year'].astype(str), format='%d/%m/%Y')
# converting gender to binary
users["Gender"] = users["Gender"].apply(lambda x: 1 if x == 'Male' else 0)


#cards
cards = pd.read_csv('{}sd254_cards.csv'.format(loc))
cards = cards.drop(['Year PIN last Changed','Card on Dark Web'], axis = 1)
#changing card number to have CN to avoid any issues
cards['Card Number'] = "CN"+cards['Card Number'].astype(str)
#changing CVV to string to avoid any issues security code = SC
cards['CVV'] = "SC"+cards['CVV'].astype(str)

# amount to numerical
cards['Credit Limit'] = pd.to_numeric(cards['Credit Limit'].str.replace('$',''))




Cleaning cards. changing expiry dates to assume they are all on the first of the month and we make the assumption that the accounts are all opened on the first. this is the only other date assumption made.

In [7]:
cards['Expires'] = pd.to_datetime(cards['Expires'], format='%m/%Y') + pd.offsets.MonthEnd(1)
cards['Acct Open Date'] = pd.to_datetime('01/' + cards['Acct Open Date'], format='%d/%m/%Y')

transaction feature:
1. creating point in time features: time of day, day of week
2. number of active cards a user has at a single point in time. a user can have multiple cards for a single account, and can have multiple accounts.

In [8]:
# creating datetime feature
transactions['date_time'] = transactions['Year'].astype(str) + '-' + transactions['Month'].astype(str) + '-' + transactions['Day'].astype(str) + ' ' + transactions['Time'].astype(str)
transactions['date_time'] = pd.to_datetime(transactions['date_time'])
# day of the weeks
transactions['DoW'] = (transactions['date_time'].dt.dayofweek).astype('int16')
# number of minutes into the day
transactions['minute'] = ((transactions['date_time'].dt.hour * 60 ) + transactions['date_time'].dt.minute).astype('float16')


In [11]:
# merging transactions and cards to id the number of active cards for a users at any given transactions
transactions_cards = pd.merge(transactions[['User','PK','date_time']],cards[['User','Acct Open Date','Expires', 'Card Number', 'Cards Issued',
                                                                             'CARD INDEX', 'Card Brand',  'Card Type', 'Credit Limit']], left_on = ['User'], right_on = ['User'], how = 'left')
transactions_cards = transactions_cards[transactions_cards['date_time'].between(transactions_cards['Acct Open Date'],transactions_cards['Expires'])]

#transactions_cards['Credit Limit'] = pd.to_numeric(transactions_cards['Credit Limit'].str.replace('$',''))
#calculating the number of active account at any given moment for every payment by grouping by the pk which is joined on user

sub_groups = transactions_cards.groupby('PK').agg(number_active_cards =('Card Number', 'nunique'),
                               number_active_accounts =('Cards Issued', 'sum')).reset_index()

# an inner join was chosen as a left join return some null values. these null values were from where there were payments made BEFORE any account was open for a user. this is an error
# this error will be reported in the thesis
transactions = transactions.merge(sub_groups, on = 'PK', how = 'inner')
del transactions_cards, sub_groups


In [12]:
transactions

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,Zip,MCC,Errors?,Is Fraud?,PK,date_time,DoW,minute,number_active_cards,number_active_accounts
0,0,0,2002,9,1,06:21,134.09,Swipe Transaction,MN3527213246127876953,La Verne,...,Z91750.0,M5300,NaN,No,0,2002-09-01 06:21:00,6,381.0,1,2
1,0,0,2002,9,1,06:42,38.48,Swipe Transaction,MN-727612092139916043,Monterey Park,...,Z91754.0,M5411,NaN,No,1,2002-09-01 06:42:00,6,402.0,1,2
2,0,0,2002,9,2,06:22,120.34,Swipe Transaction,MN-727612092139916043,Monterey Park,...,Z91754.0,M5411,NaN,No,2,2002-09-02 06:22:00,0,382.0,1,2
3,0,0,2002,9,2,17:45,128.95,Swipe Transaction,MN3414527459579106770,Monterey Park,...,Z91754.0,M5651,NaN,No,3,2002-09-02 17:45:00,0,1065.0,1,2
4,0,0,2002,9,3,06:23,104.71,Swipe Transaction,MN5817218446178736267,La Verne,...,Z91750.0,M5912,NaN,No,4,2002-09-03 06:23:00,1,383.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385537,1999,1,2020,2,27,22:23,-54.00,Chip Transaction,MN-5162038175624867091,Merrimack,...,Z3054.0,M5541,NaN,No,24386895,2020-02-27 22:23:00,3,1343.0,1,1
24385538,1999,1,2020,2,27,22:24,54.00,Chip Transaction,MN-5162038175624867091,Merrimack,...,Z3054.0,M5541,NaN,No,24386896,2020-02-27 22:24:00,3,1344.0,1,1
24385539,1999,1,2020,2,28,07:43,59.15,Chip Transaction,MN2500998799892805156,Merrimack,...,Z3054.0,M4121,NaN,No,24386897,2020-02-28 07:43:00,4,463.0,1,1
24385540,1999,1,2020,2,28,20:10,43.12,Chip Transaction,MN2500998799892805156,Merrimack,...,Z3054.0,M4121,NaN,No,24386898,2020-02-28 20:10:00,4,1210.0,1,1


merging in the user information to make age at time of payment.

In [13]:
transactions = pd.merge(transactions,users[['User', 'DOB','Gender','Zipcode','Per Capita Income - Zipcode']], left_on= ['User'], right_on = ['User'])
transactions['age at time'] = ((transactions['date_time'] - transactions['DOB']).dt.days).astype(int) // 365
transactions

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,date_time,DoW,minute,number_active_cards,number_active_accounts,DOB,Gender,Zipcode,Per Capita Income - Zipcode,age at time
0,0,0,2002,9,1,06:21,134.09,Swipe Transaction,MN3527213246127876953,La Verne,...,2002-09-01 06:21:00,6,381.0,1,2,1966-11-01,0,Z91750,29278,35
1,0,0,2002,9,1,06:42,38.48,Swipe Transaction,MN-727612092139916043,Monterey Park,...,2002-09-01 06:42:00,6,402.0,1,2,1966-11-01,0,Z91750,29278,35
2,0,0,2002,9,2,06:22,120.34,Swipe Transaction,MN-727612092139916043,Monterey Park,...,2002-09-02 06:22:00,0,382.0,1,2,1966-11-01,0,Z91750,29278,35
3,0,0,2002,9,2,17:45,128.95,Swipe Transaction,MN3414527459579106770,Monterey Park,...,2002-09-02 17:45:00,0,1065.0,1,2,1966-11-01,0,Z91750,29278,35
4,0,0,2002,9,3,06:23,104.71,Swipe Transaction,MN5817218446178736267,La Verne,...,2002-09-03 06:23:00,1,383.0,1,2,1966-11-01,0,Z91750,29278,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385537,1999,1,2020,2,27,22:23,-54.00,Chip Transaction,MN-5162038175624867091,Merrimack,...,2020-02-27 22:23:00,3,1343.0,1,1,1998-11-01,0,Z3054,32325,21
24385538,1999,1,2020,2,27,22:24,54.00,Chip Transaction,MN-5162038175624867091,Merrimack,...,2020-02-27 22:24:00,3,1344.0,1,1,1998-11-01,0,Z3054,32325,21
24385539,1999,1,2020,2,28,07:43,59.15,Chip Transaction,MN2500998799892805156,Merrimack,...,2020-02-28 07:43:00,4,463.0,1,1,1998-11-01,0,Z3054,32325,21
24385540,1999,1,2020,2,28,20:10,43.12,Chip Transaction,MN2500998799892805156,Merrimack,...,2020-02-28 20:10:00,4,1210.0,1,1,1998-11-01,0,Z3054,32325,21


In [14]:
# relationships previously mentioned
relationships = {'User':['User'],
                     'User-Merchant':['User','Merchant Name'],
                     'User-Card':['User','Card'],
                     'User-MCC':['User','MCC'],
                     'Merchant':['Merchant Name'], }
# looping through each relationship to create errorr counter (cumlative count of number of errors)
list_of_rejection = list()
for k,v in relationships.items():
    transactions = transactions.sort_values(v+['date_time'], ascending=True)
    result = transactions.groupby(v)['Amount'].agg(['cumcount'])
    result.columns = ['{} error_counter'.format(k)]

    transactions = pd.concat([transactions,result], axis=1)
    transactions['{} error_counter'.format(k)] = (transactions['{} error_counter'.format(k)] + 1).astype(int)
    list_of_rejection.append('{} error_counter'.format(k))
del result, k, v
transactions
# list of created relationships
list_of_rejection = ['User error_counter', 'User-Merchant error_counter', 'User-Card error_counter', 'User-MCC error_counter',
       'Merchant error_counter']

creating two dataframes. The first containing all payments that have gone through and those that were either 'returns' (indicated by a negative amount) or a rejected payment (indicated by an entry in the 'error?' column)

In [15]:
transactions_non_complete = transactions[(transactions['Errors?'].str.len() > 0)]
transactions_non_complete['previous_error'] = 1

<ipython-input-15-82a5b6ce84ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_non_complete['previous_error'] = 1


Creating the trailing payment for each error flag

In [16]:
transactions_non_complete_original = transactions_non_complete.copy()

transactions_non_complete1 = transactions_non_complete.copy()
transactions_non_complete1[list_of_rejection] += 1

transactions_non_complete2 = transactions_non_complete1.copy()
transactions_non_complete2[list_of_rejection] += 1

transactions_non_complete3 = transactions_non_complete2.copy()
transactions_non_complete3[list_of_rejection] += 1

transactions_non_complete = pd.concat([transactions_non_complete,transactions_non_complete1, transactions_non_complete2, transactions_non_complete3])
transactions_non_complete
del transactions_non_complete1, transactions_non_complete2, transactions_non_complete3



Joining the error counter back to the transactions table.

In [17]:
# user
transactions = transactions.merge(transactions_non_complete[['User','User error_counter', 'previous_error']].drop_duplicates().rename(columns={'previous_error': 'User - previous_error'}),
                                  left_on = ['User','User error_counter'], right_on = ['User','User error_counter'],
                                  how = 'left')
# user merchant
transactions = transactions.merge(transactions_non_complete[['User','Merchant Name','User-Merchant error_counter', 'previous_error']].drop_duplicates().rename(columns={'previous_error': 'User-Merchant - previous_error'}),
                                  left_on = ['User', 'Merchant Name' ,'User-Merchant error_counter'], right_on = ['User','Merchant Name','User-Merchant error_counter'],
                                  how = 'left')

# user card
transactions = transactions.merge(transactions_non_complete[['User','Card','User-Card error_counter', 'previous_error']].drop_duplicates().rename(columns={'previous_error': 'User-Card - previous_error'}),
                                  left_on = ['User', 'Card' ,'User-Card error_counter'], right_on = ['User','Card', 'User-Card error_counter'],
                                  how = 'left')
# user MCC
transactions = transactions.merge(transactions_non_complete[['User','MCC','User-MCC error_counter', 'previous_error']].drop_duplicates().rename(columns={'previous_error': 'User-MCC - previous_error'}),
                                  left_on = ['User', 'MCC' ,'User-MCC error_counter'], right_on = ['User','MCC', 'User-MCC error_counter'],
                                  how = 'left')
#merchant
transactions = transactions.merge(transactions_non_complete[['Merchant Name','Merchant error_counter', 'previous_error']].drop_duplicates().rename(columns={'previous_error': 'Merchant - previous_error'}),
                                  left_on = ['Merchant Name','Merchant error_counter'], right_on = ['Merchant Name','Merchant error_counter'],
                                  how = 'left')
transactions


,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,User error_counter,User-Merchant error_counter,User-Card error_counter,User-MCC error_counter,Merchant error_counter,User - previous_error,User-Merchant - previous_error,User-Card - previous_error,User-MCC - previous_error,Merchant - previous_error
0,1523,0,2013,3,27,15:35,12.62,Swipe Transaction,MN-1000080909058489971,Judson,...,3004,1,3004,50,1,NaN,NaN,NaN,NaN,NaN
1,369,0,2020,1,20,11:40,21.15,Chip Transaction,MN-1000080909058489971,Judson,...,74,1,74,1,2,NaN,NaN,NaN,NaN,NaN
2,369,0,2020,1,24,11:31,26.14,Chip Transaction,MN-1000080909058489971,Judson,...,91,2,91,2,3,NaN,NaN,NaN,NaN,NaN
3,369,1,2020,2,12,11:35,17.89,Chip Transaction,MN-1000080909058489971,Judson,...,177,3,49,3,4,NaN,NaN,NaN,NaN,NaN
4,182,0,2006,5,15,09:11,5.27,Swipe Transaction,MN-10001911495395719,Evensville,...,742,1,742,52,1,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385537,1320,3,2015,12,24,11:40,338.18,Chip Transaction,MN999682974109284083,Bluffton,...,12445,5,2202,5,9,NaN,NaN,NaN,NaN,NaN
24385538,1298,2,2017,11,1,16:55,364.73,Swipe Transaction,MN999682974109284083,Pensacola,...,15407,5,4595,5,10,NaN,NaN,NaN,NaN,NaN
24385539,1298,2,2018,5,16,16:35,350.95,Swipe Transaction,MN999682974109284083,Pensacola,...,15926,6,5114,6,11,NaN,NaN,NaN,NaN,NaN
24385540,1298,2,2018,8,20,16:57,386.91,Swipe Transaction,MN999682974109284083,Pensacola,...,16160,7,5348,7,12,1.0,NaN,1.0,NaN,NaN


In [18]:
#inspecting average previous errors
transactions.iloc[:,-5:].fillna(0).mean()

User - previous_error             0.060124
User-Merchant - previous_error    0.058211
User-Card - previous_error        0.060294
User-MCC - previous_error         0.059174
Merchant - previous_error         0.060203
dtype: float64

**IMPORTANT: USER RELATIONSHIP**
An analyst CANNOT see the future and therefore all features must be created in a manner that one would see payments coming naturally into the system. For example any 'mean' payment must not include any payments information made after the payment in question. Within my training with working within fraud detection this has been referred to as 'pre-bang' and 'post-bang' information. All features will such be deterived 'pre-bang'.

Within this payment system there are **four** easily defined relationships to observe.
1. **USER** - This is defined as all the users interactions within the system regardless of any other conditioning factor.
2. **MERCHANT** - this is defined as the all the merchants interactions within the system regardless of the user.
3. **USER & MERCHANT** - this is defined by each individual users interaction within the system with a particular merchant. This allows for a more clear understanding of a user and this merchant
4. **USER & CARD** - Users within this system may have different cards for different uses, and it is possible only a certain card may be compromised.
5. **USER & MCC** - Users within the system interact with different MCC in different patterns.


**NOTE:** additional depth user-merchant-card, user-mcc-card were considered however were rejected as feature-engineering was not the primary focus of this project.






In [19]:
# relationships previously mentioned
relationships = {'User':['User'],
                     'User-Merchant':['User','Merchant Name'],
                     'User-Card':['User','Card'],
                     'User-MCC':['User','MCC'],
                     'Merchant':['Merchant Name'], }

calculating:
1. cumsum
2. cumcount
3. cum-mean
4. cum-std

rolling:

1. rolling mean and std at 3 window
2. rolling mean and std at 7 window


**note: all money amount are being treated as positives**

In [20]:
# keeping an un_mutated version of the amount
transactions['Amount_og'] = transactions['Amount']
# changing 'returns' (negative amounts) to positives. they are still information and should be kept
transactions['Amount'] = abs(transactions['Amount'])
for k,v in relationships.items():
    transactions = transactions.sort_values(v+['date_time'], ascending=True)
    result = transactions.groupby(v)['Amount'].agg(['cumsum', 'cumcount'])
    result.columns = ['{} CS'.format(k), '{} CC'.format(k)]

    transactions = pd.concat([transactions,result], axis=1)
    transactions['{} CC'.format(k)] = transactions['{} CC'.format(k)] + 1



    transactions['{} CM'.format(k)] = transactions['{} CS'.format(k)] / transactions['{} CC'.format(k)]


    transactions['{} CSTD'.format(k)] = transactions.groupby(v)['Amount'].expanding().std().reset_index(0)['Amount'].tolist()
    transactions['{} CSTD'.format(k)] = transactions['{} CSTD'.format(k)].fillna(0)

    #rolling means
    transactions['{} CM3'.format(k)] =transactions.groupby(v)['Amount'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True).tolist()
    transactions['{} CSTD3'.format(k)] =transactions.groupby(v)['Amount'].rolling(window=3, min_periods=1).std().reset_index(0, drop=True).fillna(0).tolist()
    transactions['{} CSTD7'.format(k)] = transactions.groupby(v)['Amount'].rolling(window=7, min_periods=1).std().reset_index(0, drop=True).fillna(0).tolist()
    transactions['{} CM7'.format(k)] = transactions.groupby(v)['Amount'].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True).tolist()
del result, k, v
transactions['Amount'] = transactions['Amount_og']
transactions = transactions.drop('Amount_og',axis = 1)


In [21]:
transactions

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,User-MCC CSTD7,User-MCC CM7,Merchant CS,Merchant CC,Merchant CM,Merchant CSTD,Merchant CM3,Merchant CSTD3,Merchant CSTD7,Merchant CM7
0,1523,0,2013,3,27,15:35,12.62,Swipe Transaction,MN-1000080909058489971,Judson,...,1.759493,11.821429,12.62,1,12.620000,0.000000,12.620000,0.000000,0.000000,12.620000
1,369,0,2020,1,20,11:40,21.15,Chip Transaction,MN-1000080909058489971,Judson,...,0.000000,21.150000,33.77,2,16.885000,6.031621,16.885000,6.031621,6.031621,16.885000
2,369,0,2020,1,24,11:31,26.14,Chip Transaction,MN-1000080909058489971,Judson,...,3.528463,23.645000,59.91,3,19.970000,6.836805,19.970000,6.836805,6.836805,19.970000
3,369,1,2020,2,12,11:35,17.89,Chip Transaction,MN-1000080909058489971,Judson,...,4.155121,21.726667,77.80,4,19.450000,5.678280,21.726667,4.155121,5.678280,19.450000
4,182,0,2006,5,15,09:11,5.27,Swipe Transaction,MN-10001911495395719,Evensville,...,53.858555,41.007143,5.27,1,5.270000,0.000000,5.270000,0.000000,0.000000,5.270000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385537,1320,3,2015,12,24,11:40,338.18,Chip Transaction,MN999682974109284083,Bluffton,...,34.246980,351.806000,3150.43,9,350.047778,47.206958,336.953333,28.429855,51.014806,356.298571
24385538,1298,2,2017,11,1,16:55,364.73,Swipe Transaction,MN999682974109284083,Pensacola,...,57.727209,351.226000,3515.16,10,351.516000,44.748664,336.946667,28.420078,47.296038,351.005714
24385539,1298,2,2018,5,16,16:35,350.95,Swipe Transaction,MN999682974109284083,Pensacola,...,51.632908,351.180000,3866.11,11,351.464545,42.452653,351.286667,13.278201,47.101808,349.597143
24385540,1298,2,2018,8,20,16:57,386.91,Swipe Transaction,MN999682974109284083,Pensacola,...,49.030676,356.284286,4253.02,12,354.418333,41.750296,367.530000,18.142778,39.492172,363.871429


**Tester Payment Indicator:**

many times fraudsters will test to see if a small payment amount will work, therefore first-time payments within any of the defined relationships will be flagged. A tester-payment window will be created, this window will allow for the following payments after a tester payment to be flagged as high-risk.

As an analyst who does not know what threshold is best a range will be set.

In [22]:
cuml_counts = [item for item in transactions.columns.tolist() if item[-3:] == ' CC']
cuml_counts

['User CC', 'User-Merchant CC', 'User-Card CC', 'User-MCC CC', 'Merchant CC']

In [23]:
# giving up on dynamic coding. trying hard code for relationships. christ fml
tester_payment_range = [1,5,10,20]
tester_range = list()
for x in tester_payment_range:
    # flagging payments below the threshold that are also a CC 1 for any relationship
    transactions['tester_payment_{}'.format(x)] = np.where(((transactions[cuml_counts] == 1).any(axis=1)) & (transactions['Amount'].between(0,x)),1,0)
    tester_range.append('tester_payment_{}'.format(x))

In [24]:
#transactions2 = transactions[transactions['User']==2]
# relationships previously mentioned
relationships = {'User':['User'],
                     'User-Merchant':['User','Merchant Name'],
                     'User-Card':['User','Card'],
                     'User-MCC':['User','MCC'],
                     'Merchant':['Merchant Name'],
                  }

test_payment = [item for item in transactions.columns.tolist() if 'tester_payment_' in item]
lister_rela = list()
for k,v in relationships.items():
  transactions = transactions.sort_values(v+['date_time'], ascending=True)
  for x in test_payment:
    tester_payment_temp = transactions[(transactions['{} CC'.format(k)]==1) & transactions[x]==1][v]
    tester_payment_temp['{}_lag_{}'.format(k,x)] = 1
    tester_payment_temp2 = tester_payment_temp.copy()
    tester_payment_temp0 = tester_payment_temp.copy()
    tester_payment_temp0['{} CC'.format(k)] = 1
    tester_payment_temp['{} CC'.format(k)] = 2
    tester_payment_temp2['{} CC'.format(k)] = 3
    tester_payment_temp = pd.concat([tester_payment_temp, tester_payment_temp2, tester_payment_temp0])
    del tester_payment_temp2, tester_payment_temp0
    print('{}, {}'.format(k,x))
    transactions = transactions.merge(tester_payment_temp, left_on = v + ['{} CC'.format(k)], right_on = v + ['{} CC'.format(k)], how = 'left')
    transactions['{}_lag_{}'.format(k,x)] = transactions['{}_lag_{}'.format(k,x)].fillna(value=0)
    lister_rela.append('{}_lag_{}'.format(k,x))

transactions = transactions.drop(test_payment, axis = 1)
transactions

#filtered_rows = transactions2[(transactions2[transactions2.columns[transactions2.columns.str.contains('_lag_')]] == 1).any(axis=1)]
#filtered_rows[keep].to_csv('{}inspect_small.csv'.format(loc))


User, tester_payment_1
User, tester_payment_5
User, tester_payment_10
User, tester_payment_20
User-Merchant, tester_payment_1
User-Merchant, tester_payment_5
User-Merchant, tester_payment_10
User-Merchant, tester_payment_20
User-Card, tester_payment_1
User-Card, tester_payment_5
User-Card, tester_payment_10
User-Card, tester_payment_20
User-MCC, tester_payment_1
User-MCC, tester_payment_5
User-MCC, tester_payment_10
User-MCC, tester_payment_20
Merchant, tester_payment_1
Merchant, tester_payment_5
Merchant, tester_payment_10
Merchant, tester_payment_20


,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,User-Card_lag_tester_payment_10,User-Card_lag_tester_payment_20,User-MCC_lag_tester_payment_1,User-MCC_lag_tester_payment_5,User-MCC_lag_tester_payment_10,User-MCC_lag_tester_payment_20,Merchant_lag_tester_payment_1,Merchant_lag_tester_payment_5,Merchant_lag_tester_payment_10,Merchant_lag_tester_payment_20
0,1523,0,2013,3,27,15:35,12.62,Swipe Transaction,MN-1000080909058489971,Judson,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,369,0,2020,1,20,11:40,21.15,Chip Transaction,MN-1000080909058489971,Judson,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,369,0,2020,1,24,11:31,26.14,Chip Transaction,MN-1000080909058489971,Judson,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,369,1,2020,2,12,11:35,17.89,Chip Transaction,MN-1000080909058489971,Judson,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,182,0,2006,5,15,09:11,5.27,Swipe Transaction,MN-10001911495395719,Evensville,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385537,1320,3,2015,12,24,11:40,338.18,Chip Transaction,MN999682974109284083,Bluffton,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24385538,1298,2,2017,11,1,16:55,364.73,Swipe Transaction,MN999682974109284083,Pensacola,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24385539,1298,2,2018,5,16,16:35,350.95,Swipe Transaction,MN999682974109284083,Pensacola,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24385540,1298,2,2018,8,20,16:57,386.91,Swipe Transaction,MN999682974109284083,Pensacola,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# adding payments back in that had errors.
transactions = pd.concat([transactions_non_complete_original,transactions])



In [26]:
# relationships previously mentioned
relationships = {'User':['User'],
                     'User-Merchant':['User','Merchant Name'],
                     'User-Card':['User','Card'],
                     'User-MCC':['User','MCC'],
                     'Merchant':['Merchant Name'], }

rolling_payments = list()
for k,v in relationships.items():
    payment_small = transactions[v + ['date_time']]
    payment_small['Occurrences'] = 1
    payment_small = payment_small.groupby(v + ['date_time'])['Occurrences'].sum().reset_index()
    minutes = [1,10]
    payment_small = payment_small.set_index('date_time')
    for x in minutes:
        seconds = x * 60
        payment_small['{} occurance {} mins'.format(k,x)] = payment_small.groupby(v)['Occurrences'].rolling('{}s'.format(seconds)).sum().reset_index()['Occurrences'].tolist()
        rolling_payments.append('{} occurance {} mins'.format(k,x))
    payment_small = payment_small.reset_index()
    payment_small = payment_small.drop(columns = 'Occurrences' )
    transactions = transactions.merge(payment_small, on = v + ['date_time'], how = 'left')
del payment_small

transactions

<ipython-input-26-b260f8ca042f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_small['Occurrences'] = 1
<ipython-input-26-b260f8ca042f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_small['Occurrences'] = 1
<ipython-input-26-b260f8ca042f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,User occurance 1 mins,User occurance 10 mins,User-Merchant occurance 1 mins,User-Merchant occurance 10 mins,User-Card occurance 1 mins,User-Card occurance 10 mins,User-MCC occurance 1 mins,User-MCC occurance 10 mins,Merchant occurance 1 mins,Merchant occurance 10 mins
0,1490,0,2000,10,31,13:40,59.79,Swipe Transaction,MN-100064907641264139,Tacoma,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,1490,2,2010,4,30,12:34,19.98,Swipe Transaction,MN-100064907641264139,Tacoma,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,1490,2,2010,11,15,11:39,42.47,Swipe Transaction,MN-100064907641264139,Tacoma,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,1490,4,2013,4,17,04:28,20.72,Swipe Transaction,MN-100064907641264139,Tacoma,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,1283,6,2019,9,7,12:46,17.89,Chip Transaction,MN-100064907641264139,Bronx,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24773947,1320,3,2015,12,24,11:40,338.18,Chip Transaction,MN999682974109284083,Bluffton,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
24773948,1298,2,2017,11,1,16:55,364.73,Swipe Transaction,MN999682974109284083,Pensacola,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
24773949,1298,2,2018,5,16,16:35,350.95,Swipe Transaction,MN999682974109284083,Pensacola,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
24773950,1298,2,2018,8,20,16:57,386.91,Swipe Transaction,MN999682974109284083,Pensacola,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [27]:
transactions.memory_usage(deep=True).sum() / 1024**2

32722.726139068604

Additional Features:
- below will add Fraud risks
- will standardise all inputs that require it
- This section will one-hot encode needed variables


In [28]:
# maybe deleted :_)
df = transactions
df2 = pd.read_csv("{}sd254_cards.csv".format(loc))
# converting is fraud to binary
df["Is Fraud?"] = df["Is Fraud?"].apply(lambda x: 1 if x == 'Yes' else 0)
# converting gender to binary
df["Gender"] = df["Gender"].apply(lambda x: 1 if x == 'Male' else 0)
# filling na with online (when missing value is online)
df['Merchant State'] = df['Merchant State'].fillna('Online')
# creating a primary key for the users card to avoid confusion
df['user_card'] = df['User'].astype(str)+'_'+df['Card'].astype(str)
# ensuring all missing values have been filled
df = df.fillna(0)

# merging in Card details that were not included in the first feature engineering section
df = df.merge(df2[['User','CARD INDEX', 'Card Brand',  'Card Type', 'Credit Limit']],
         left_on = ['User','Card',], right_on = ['User','CARD INDEX'],
         how = 'left').drop('CARD INDEX',axis = 1)
df['Credit Limit'] = pd.to_numeric(df['Credit Limit'].str.replace('$',''))

df3 = pd.read_csv("{}sd254_users.csv".format(loc))[['FICO Score','Total Debt','State']]
df3['User'] = df3.index
df = df.merge(df3, left_on = 'User', right_on ='User', how = 'left')
df['Total Debt'] = pd.to_numeric(df['Total Debt'].str.replace('$',''))

del df2, df3

Fraud rate is the observed number of fraud cases divided by the total number of observations in the area

In [33]:
# converting is fraud to binary
transactions["Is Fraud?"] = transactions["Is Fraud?"].apply(lambda x: 1 if x == 'Yes' else 0)


def fraud_rate(change_df, observ_df, vars,object_var):
  """
  Calculates fraud rate for specified variables based on a reference DataFrame.

  Args:
    target_df: The DataFrame to which fraud rate will be added.
    reference_df: The DataFrame used to calculate fraud rates.
    group_vars: A list of column names to group by when calculating fraud rates.
    target_var: The name of the column representing the target variable (e.g., 'Is Fraud?').

  Returns:
    The target DataFrame with fraud rate columns added.
  """
  for x in vars:
    rate = observ_df.groupby(x)[object_var].mean().to_dict()
    change_df['FR: {}'.format(x)] = change_df[x].map(rate)
    change_df['FR: {}'.format(x)] = change_df['FR: {}'.format(x)].fillna(0)
  return change_df


def contin_scaler(df,columns):
  """
  Performs min-max scaling on specified columns of a DataFrame.

  Args:
    df: The pandas DataFrame to be scaled.
    columns: A list of column names to be min-max scaled.

  Returns:
    A tuple containing:
      - The scaled DataFrame.
      - A dictionary of scalers for reversing the scaling.
  """
  from sklearn import preprocessing
  contin_dict = {}
  for x in columns:
    scaler = preprocessing.MinMaxScaler()
    df[x] = scaler.fit_transform(np.array(df[x]).reshape(-1, 1))
    contin_dict[x] = scaler
  return df, contin_dict

def rev_contin_scaler(df,dicts):
  """
  Reverses min-max scaling on specified columns of a DataFrame.

  Args:
    df: The pandas DataFrame to be reversed scaled.
    scalers: A dictionary of MinMaxScaler objects, as returned by min_max_scale.

  Returns:
    The DataFrame with min-max scaling reversed on specified columns.
  """
  for x in dicts:
    if x in df.columns:
      scaler_temp = dicts[x]
      df[x] = scaler_temp.inverse_transform(np.array(df[x]).reshape(-1, 1))
  return

def one_hot(df,columns):
  """
  Performs one-hot encoding on specified columns of a DataFrame.

  Args:
    df: The pandas DataFrame to be encoded.
    columns: A list of column names to be one-hot encoded.

  Returns:
    A tuple containing:
      - The one-hot encoded DataFrame.
      - A dictionary to reverse the one-hot encoding.
  """
  count = 0
  dict_oh = {}
  for x in columns:
    count = count +1
    one_hot_encoded = pd.get_dummies(df[x]).astype(int).add_prefix('OH{}: '.format(count))
    # Concatenate the one-hot encoded columns with the original DataFrame
    dict_oh['OH{}:'.format(count)] = x
    df = pd.concat([df, one_hot_encoded], axis=1).drop(x,axis =1)
  return df, dict_oh

def rev_one_hot(df,dicts):
   """
  Reverses one-hot encoding on a DataFrame.

  Args:
    df: The pandas DataFrame with one-hot encoded columns.
    encoding_dict: A dictionary mapping original column names to lists of
                   corresponding one-hot encoded column names, as returned
                   by one_hot_encode.

  Returns:
    The DataFrame with one-hot encoding reversed.
  """
   for x in dicts:
    print(x)
    cols = [item for item in df.columns if item[0:4] == x]
    #print(cols)
    old_names = [string[5:] for string in cols]
    holder = df[cols]
    holder.columns = old_names
    holder = pd.from_dummies((holder))
    holder.columns = [dicts[x]]
    df = pd.concat([df, holder], axis=1).drop(cols,axis =1)
   return df

# clustering and one-hot encoding errors
df['Error - Bad input'] = np.where(df['Errors?'].fillna('No error').str.contains('Bad '),1,0)
df['Error - Insuf bal'] = np.where(df['Errors?'].fillna('No error').str.contains('Insufficient '),1,0)
df['Error - Tech Glitch'] = np.where(df['Errors?'].fillna('No error').str.contains('Technical'),1,0)
df = df.drop('Errors?', axis = 1)

#creating if in country (USA)
df['Merchant in Counry'] = np.where(len(df['Merchant State'])== 2, 1, 0)

def merch_mapping(input):
  """
  Maps merchant locations to risk categories.

  Args:
    location: The merchant location string.

  Returns:
    A string indicating the risk category of the location.
  """
  world_high_risk = ['TUVALU','ALGERIA','HAITI','FIJI',
                     'NIGERIA','TURKEY','ITALY']
  if input == 'OH':
    return 'Ohio'
  elif len(input) == 2:
    return 'US'
  elif input.upper() in world_high_risk:
    return 'high_risk'
  elif input == 'Online':
    return input
  elif len(input) > 2:
    return 'world_non_us'
  else:
    return 'ERROR'

df['Merchant State2'] = df['Merchant State'].apply(merch_mapping)
df['User State2'] = df['State'].apply(merch_mapping)
pd.DataFrame(df.groupby('Merchant State2')['Is Fraud?'].mean()).sort_values('Is Fraud?', ascending = False)
df, dict_oh = one_hot(df, ['Use Chip','Card Brand', 'Card Type','Merchant State2','User State2'])
df['Merchant State2'] = df['Merchant State'].apply(merch_mapping)
df['User State2'] = df['State'].apply(merch_mapping)


df["Fraud2"] = df["Is Fraud?"]





# Apply the mapping function to the MCC column
#df, dict_oh = one_hot(df, ['Use Chip','Card Brand', 'Card Type','Merchant State2','User State2'])
df['MCC'] = df['MCC'].str[1:].astype(int)

df['OH2: Agricultural Services'] = np.where((df['MCC'] >= 1) & (df['MCC'] <= 1500),1,0)
df['OH2: Contracted Services'] = np.where((df['MCC'] >= 1500) & (df['MCC'] <= 4000),1,0)
df['OH2: Transportation Services'] = np.where((df['MCC'] >= 4000) & (df['MCC'] <= 4800),1,0)
df['OH2: Utility Services'] = np.where((df['MCC'] >= 4800) & (df['MCC'] <= 5000),1,0)
df['OH2: Retail Outlet Services'] = np.where((df['MCC'] >= 5000) & (df['MCC'] <= 5600),1,0)
df['OH2: Clothing Stores'] = np.where((df['MCC'] >= 5600) & (df['MCC'] <= 5700),1,0)
df['OH2: Miscellaneous Stores'] = np.where((df['MCC'] >= 5700) & (df['MCC'] <= 7300),1,0)
df['OH2: Business Services'] = np.where((df['MCC'] >= 7300) & (df['MCC'] <= 8000),1,0)
df['OH2: Professional Services and Membership Organizations'] = np.where((df['MCC'] >= 8000) & (df['MCC'] <= 9000),1,0)
df['OH2: Government Services'] = np.where((df['MCC'] >= 9000) & (df['MCC'] <= 10000),1,0)



# getting the fraud rate for the respective places
cleaned_list1 = ['MCC', 'Merchant City','Merchant State','Zip', 'Zipcode']
df = fraud_rate(df, df, cleaned_list1, 'Is Fraud?')

cleaned_list2 = ['Per Capita Income - Zipcode', 'DoW', 'minute', 'number_active_cards', 'number_active_accounts', 'Amount','age at time',  # random varaiables
                 'User error_counter',  'User-Merchant error_counter',  'User-Card error_counter',  'User-MCC error_counter', 'Merchant error_counter', # user error counts = a rolling count of pprevious errors
                 'User - previous_error', 'User-Merchant - previous_error', 'User-Card - previous_error', 'User-MCC - previous_error', 'Merchant - previous_error',  # indicator if the previous payments were errors
                 'User CS', 'User CC', 'User CM', 'User CSTD', 'User CM3', 'User CSTD3', 'User CSTD7', 'User CM7', # user relationship
                 'User-Merchant CS', 'User-Merchant CC', 'User-Merchant CM', 'User-Merchant CSTD', 'User-Merchant CM3', 'User-Merchant CSTD3', 'User-Merchant CSTD7', 'User-Merchant CM7', # user- merchant relationship
                 'User-Card CS', 'User-Card CC', 'User-Card CM', 'User-Card CSTD', 'User-Card CM3', 'User-Card CSTD3', 'User-Card CSTD7', 'User-Card CM7', # user-card relationship
                 'User-MCC CS', 'User-MCC CC', 'User-MCC CM', 'User-MCC CSTD', 'User-MCC CM3', 'User-MCC CSTD3', 'User-MCC CSTD7', 'User-MCC CM7', # user - mcc relationship
                 'Merchant CS', 'Merchant CC', 'Merchant CM', 'Merchant CSTD', 'Merchant CM3', 'Merchant CSTD3', 'Merchant CSTD7', 'Merchant CM7', # merchant relationship
                 'User_lag_tester_payment_1', 'User_lag_tester_payment_5', 'User_lag_tester_payment_10', 'User_lag_tester_payment_20', # user test lag payments
                 'User-Merchant_lag_tester_payment_1', 'User-Merchant_lag_tester_payment_5', 'User-Merchant_lag_tester_payment_10', 'User-Merchant_lag_tester_payment_20', # user merchant relationship tester payments
                 'User-Card_lag_tester_payment_1', 'User-Card_lag_tester_payment_5', 'User-Card_lag_tester_payment_10', 'User-Card_lag_tester_payment_20', # user cards tester payment
                 'User-MCC_lag_tester_payment_1', 'User-MCC_lag_tester_payment_5', 'User-MCC_lag_tester_payment_10', 'User-MCC_lag_tester_payment_20', # user MCC tester payments
                 'Merchant_lag_tester_payment_1', 'Merchant_lag_tester_payment_5', 'Merchant_lag_tester_payment_10', 'Merchant_lag_tester_payment_20', # merchant tester paymetns
                 'User occurance 1 mins', 'User occurance 10 mins', # user occ in mins
                 'User-Merchant occurance 1 mins', 'User-Merchant occurance 10 mins', # user merchant occurance in mins
                 'User-Card occurance 1 mins', 'User-Card occurance 10 mins', # user card occurance in mins
                 'User-MCC occurance 1 mins', 'User-MCC occurance 10 mins', # user MCC occurance in mins
                 'Merchant occurance 1 mins','Merchant occurance 10 mins', # merchants in mins
                 'Credit Limit',
                 'FICO Score', 'Total Debt'] # credit limit

df, contin_dict = contin_scaler(df,cleaned_list2)




<ipython-input-33-edcb1cc02fca>:112: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Error - Bad input'] = np.where(df['Errors?'].fillna('No error').str.contains('Bad '),1,0)
<ipython-input-33-edcb1cc02fca>:113: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Error - Insuf bal'] = np.where(df['Errors?'].fillna('No error').str.contains('Insufficient '),1,0)
<ipython-input-33-edcb1cc02fca>:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor p

In [34]:
#export final transactions
df.to_csv('{}clean_processed_transactions.csv'.format(loc))